## TO EXTRACT FILE

In [ ]:
pip install rarfile


In [ ]:
import rarfile
import os
rar_file_path = '/content/Plant detection.rar'
destination_folder = '/content/test'
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(destination_folder)
print("Extracted files:", os.listdir(destination_folder))

## LIBRARIES 

In [ ]:
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation,Flatten, Add, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

## Setting path of train and valid data

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory=r'/content/test/Plant detection/Train',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(256, 256)
)

In [ ]:
validation_ds =keras.utils.image_dataset_from_directory(
    directory =r'/content/test/Plant detection/Test',
    labels = 'inferred',
    label_mode = 'binary',
    batch_size=32,
    image_size=(256,256)
)

## MODEL STUCTURE

In [ ]:
l2_lambda = 0.001

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu',
                 input_shape=(256, 256, 3), kernel_regularizer=l2(l2_lambda)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu',
                 kernel_regularizer=l2(l2_lambda)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu',
                 kernel_regularizer=l2(l2_lambda)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_lambda)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)

In [ ]:
model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

## Early Srtopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

## TRAIN MODEL

In [ ]:

import time
start_time = time.time()
history = model.fit(train_ds,
                    validation_data=validation_ds,
                    epochs=100,
                    batch_size=32,
                    callbacks=[early_stopping])
end_time = time.time()
training_time = end_time - start_time
print(training_time)


## Save MDOEL

In [ ]:
import joblib
joblib.dump(model, 'Plant_detection.pkl')